In [1]:
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
import ast
from statistics import mean
from PIL import Image
from collections import Counter, defaultdict

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard


In [2]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import cv2
from sklearn.model_selection import train_test_split

# Mount Google Drive
drive.mount('/content/drive')

# Base directory (adjust as needed)
base_dir = "/content/drive/MyDrive/projet_collembolles"

# Paths for CSVs and image folders
AGREED_BOXES_CSV = os.path.join(base_dir, "agreed_boxes.csv")
BACKGROUND_BOXES_CSV = os.path.join(base_dir, "random_background_boxes.csv")
IMAGES_FOLDER = os.path.join(base_dir, "data")       # Labeled images folder (uncropped)
TEST_FOLDER = os.path.join(base_dir, "datatest")       # Test images folder (already cropped)

print("AGREED_BOXES_CSV:", AGREED_BOXES_CSV)
print("BACKGROUND_BOXES_CSV:", BACKGROUND_BOXES_CSV)
print("IMAGES_FOLDER:", IMAGES_FOLDER)
print("TEST_FOLDER:", TEST_FOLDER)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AGREED_BOXES_CSV: /content/drive/MyDrive/projet_collembolles/agreed_boxes.csv
BACKGROUND_BOXES_CSV: /content/drive/MyDrive/projet_collembolles/random_background_boxes.csv
IMAGES_FOLDER: /content/drive/MyDrive/projet_collembolles/data
TEST_FOLDER: /content/drive/MyDrive/projet_collembolles/datatest


## Nombre d'avis en commun pour créé label :

In [3]:
min_avis = 4

## Créé fonction csv

In [4]:
# Le dossier où il y a les images, pour récupérer chaque txt
def info_data(txt_folder):
    data = []
    for txt_file in os.listdir(txt_folder):
        if txt_file.endswith('.txt'):
            with open(os.path.join(txt_folder, txt_file), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    avis = parts[0]
                    xc = parts[-4]
                    yc = parts[-3]
                    w = parts[-2]
                    h = parts[-1]
                    other = parts[1:-4]
                    file_id = txt_file.replace('.txt', '')
                    data.append([file_id, avis] + [xc, yc, w, h] + other)
    columns = ['id', 'avis'] + ['xc', 'yc', 'w', 'h'] + ['classe','year'] + [f'info{i+2}' for i in range(4)]
    df = pd.DataFrame(data, columns=columns)
    df.to_csv('info_data.csv', index=False)

## CLasse python image et background

In [5]:
# On définit la graine pour assurer la reproductibilité
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [6]:
class BackgroundDataset(Dataset):
    def __init__(self, images_dir, csv_file, transform=None, num_samples=1000):
        """
        Dataset qui génère des images de fond (classe 8) en évitant les bounding boxes existantes.

        Args:
            images_dir (string): Dossier contenant les images.
            csv_file (string): Chemin vers le fichier CSV contenant les informations.
            transform (callable, optional): Transformations à appliquer aux images.
            num_samples (int): Nombre de samples de background à générer.
        """
        self.images_dir = images_dir
        self.transform = transform
        self.num_samples = num_samples

        # Charger les informations du dataset original
        self.data = pd.read_csv(csv_file)
        self.boxes = self._load_boxes()
        self.image_ids = list(self.boxes.keys())  # Liste des images disponibles

    def _load_boxes(self):
        """
        Charge toutes les bounding boxes sous forme de dictionnaire {image_id: [bbox_list]}.
        """
        boxes = {}
        for _, row in self.data.iterrows():
            image_id = str(row['id'])
            bbox = [row['xc'], row['yc'], row['w'], row['h']]
            if image_id not in boxes:
                boxes[image_id] = []
            boxes[image_id].append(bbox)
        return boxes

    def _get_random_background_patch(self, image, bboxes):
        """
        Extrait une zone aléatoire de l'image en évitant les bounding boxes existantes.
        """
        width, height = image.size
        patch_size = 822  # Taille fixe du patch

        for _ in range(10):  # Essayer 10 fois de trouver une zone correcte
            x = random.randint(0, width - patch_size)
            y = random.randint(0, height - patch_size)

            # Vérifier si la zone chevauche une bounding box
            overlaps = False
            for (xc, yc, w, h) in bboxes:
                x_min = int((xc - w / 2) * width)
                x_max = int((xc + w / 2) * width)
                y_min = int((yc - h / 2) * height)
                y_max = int((yc + h / 2) * height)

                if not (x_max < x or x_min > x + patch_size or y_max < y or y_min > y + patch_size):
                    overlaps = True
                    break

            if not overlaps:
                return image.crop((x, y, x + patch_size, y + patch_size))

        # Si aucune zone correcte n'a été trouvée après 10 essais, prendre une zone au hasard
        return image.crop((0, 0, patch_size, patch_size))

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        """
        Retourne un patch de fond et le label 8.
        """
        image_id = random.choice(self.image_ids)
        image_path = os.path.join(self.images_dir, f"{image_id}.jpg")
        image = Image.open(image_path).convert("RGB")

        bboxes = self.boxes[image_id]
        background_patch = self._get_random_background_patch(image, bboxes)

        if self.transform:
            background_patch = self.transform(background_patch)

        label = torch.tensor(8, dtype=torch.long)

        return background_patch, label


class ImageBoundingBoxDataset(Dataset):
    def __init__(self, images_dir, csv_file, transform=None, split="train", seed=42):
        """
        Args:
            images_dir (string): Dossier contenant les images.
            csv_file (string): Chemin vers le fichier CSV contenant les informations.
            transform (callable, optional): Transformations à appliquer aux images.
            split (string): "train", "val" ou "test" pour choisir le dataset.
            seed (int): Pour rendre la répartition fixe.
        """
        self.images_dir = images_dir
        self.transform = transform
        self.split = split
        self.seed = seed

        self.data = pd.read_csv(csv_file)
        self.boxes = self._create_boxes_list()

        self._split_data()

        self.class_counts = self.count_classes()

    def _create_boxes_list(self):
        """
        Crée une liste de dictionnaires où chaque boîte est une entrée unique.
        """
        boxes = []
        for _, row in self.data.iterrows():
            image_id = str(row['id'])
            bbox = [row['xc'], row['yc'], row['w'], row['h']]
            avis = row['avis']
            label = self.avis_majoritaire(avis)
            if label not in [None, 8]:
                boxes.append({
                    'image_id': image_id,
                    'bbox': bbox,
                    'label': label
                })
        return boxes

    def _split_data(self):
        """
        Effectue le split de l'ensemble de données en train, validation et test,
        et choisit le split actif.
        """
        labels = [box["label"] for box in self.boxes]
        train_data, test_data = train_test_split(self.boxes, test_size=0.4, random_state=self.seed, stratify=labels)

        # Choisir le dataset basé sur le split demandé
        if self.split == "train":
            self.data_split = train_data
        elif self.split == "test":
            self.data_split = test_data
        else:
            raise ValueError("Split must be one of ['train', 'val', 'test']")

    def count_classes(self):
        """
        Compte le nombre d'instances pour chaque classe dans le dataset actuel.
        """
        class_counts = defaultdict(int)
        for annotation in self.data_split:
            label = annotation['label']
            class_counts[label] += 1
        return dict(class_counts)


    def avis_majoritaire(self, avis, min_count=min_avis):
        """Calcule l'avis majoritaire uniquement s'il dépasse un seuil minimal."""
        parts = avis.split('_')
        count = Counter(parts)

        majoritaire, occurrences = max(count.items(), key=lambda x: x[1])

        if occurrences >= min_count:
            return int(majoritaire)

        return None


    def __len__(self):
        return len(self.data_split)


    def __getitem__(self, idx):
        """
        Retourne une image découpée selon la boîte englobante et son label.
        """
        annotation = self.data_split[idx]
        bbox = annotation['bbox']
        label = annotation['label']

        image_path = os.path.join(self.images_dir, f"{annotation['image_id']}.jpg")
        image = Image.open(image_path).convert("RGB")

        xc, yc, w, h = bbox
        x_min = int((xc - w / 2) * image.width)
        x_max = int((xc + w / 2) * image.width)
        y_min = int((yc - h / 2) * image.height)
        y_max = int((yc + h / 2) * image.height)

        cropped_image = image.crop((x_min, y_min, x_max, y_max))

        if self.transform:
            cropped_image = self.transform(cropped_image)

        label = torch.tensor(label, dtype=torch.long)

        return cropped_image, label


In [7]:
class Image_label_Dataset(Dataset):
    def __init__(self, images_dir, csv_file, fallback_csv, transform=None, split="train", seed=42):
        """
        Args:
            images_dir (string): Dossier contenant les images.
            csv_file (string): Chemin vers le fichier CSV contenant les informations principales.
            fallback_csv (string): Chemin vers le fichier CSV contenant image_id + bbox + final_label.
            transform (callable, optional): Transformations à appliquer aux images.
            split (string): "train" ou "test" pour choisir le dataset.
            seed (int): Pour rendre la répartition fixe.
        """

        self.images_dir = images_dir
        self.transform = transform
        self.split = split
        self.seed = seed

        self.data = pd.read_csv(csv_file)
        self.fallback_data = pd.read_csv(fallback_csv)
        self.boxes = self._create_boxes_list()

        self._split_data()
        self.class_counts = self.count_classes()

    def _create_boxes_list(self):
        """
        Crée une liste de dictionnaires où chaque boîte est une entrée unique.
        """
        boxes = []
        for _, row in self.data.iterrows():
            image_id = str(row['id'])
            bbox = [row['xc'], row['yc'], row['w'], row['h']]
            avis = row['avis']
            label = self.avis_majoritaire(avis)

            if label is None:
                label = self.get_fallback_label(image_id, bbox)

            if label not in [None, 8]:
                boxes.append({
                    'image_id': image_id,
                    'bbox': bbox,
                    'label': label
                })
        return boxes

    def get_fallback_label(self, image_id, bbox):
        """
        Cherche le label dans le fichier CSV de fallback si avis_majoritaire retourne None.
        """
        # Convertir la chaîne de caractères du 'bbox' du fichier fallback en tuple
        self.fallback_data['bbox_tuple'] = self.fallback_data['bbox'].apply(lambda x: ast.literal_eval(x))

        # Arrondir le bbox à 5 décimales pour correspondre au format du fichier fallback
        bbox_round = tuple(round(val, 5) for val in bbox)

        # Recherche d'une correspondance
        match = self.fallback_data[(self.fallback_data['idx'] == image_id) &
                                    (self.fallback_data['bbox_tuple'] == bbox_round)]

        if not match.empty:
            return int(match.iloc[0]['final_label'])

        return None

    def _split_data(self):
        """
        Effectue le split de l'ensemble de données en train et test.
        """
        labels = [box["label"] for box in self.boxes]
        train_data, test_data = train_test_split(self.boxes, test_size=0.4, random_state=self.seed, stratify=labels)

        if self.split == "train":
            self.data_split = train_data
        elif self.split == "test":
            self.data_split = test_data
        else:
            raise ValueError("Split must be one of ['train', 'test']")

    def count_classes(self):
        """
        Compte le nombre d'instances pour chaque classe dans le dataset actuel.
        """
        class_counts = defaultdict(int)
        for annotation in self.data_split:
            label = annotation['label']
            class_counts[label] += 1
        return dict(class_counts)

    def avis_majoritaire(self, avis, min_count=4):
        """Calcule l'avis majoritaire uniquement s'il dépasse un seuil minimal."""
        parts = avis.split('_')
        count = Counter(parts)

        majoritaire, occurrences = max(count.items(), key=lambda x: x[1])

        if occurrences >= min_count:
            return int(majoritaire)

        return None

    def __len__(self):
        return len(self.data_split)

    def __getitem__(self, idx):
        """
        Retourne une image découpée selon la boîte englobante et son label.
        """
        annotation = self.data_split[idx]
        bbox = annotation['bbox']
        label = annotation['label']

        image_path = os.path.join(self.images_dir, f"{annotation['image_id']}.jpg")
        image = Image.open(image_path).convert("RGB")

        xc, yc, w, h = bbox
        x_min = int((xc - w / 2) * image.width)
        x_max = int((xc + w / 2) * image.width)
        y_min = int((yc - h / 2) * image.height)
        y_max = int((yc + h / 2) * image.height)

        cropped_image = image.crop((x_min, y_min, x_max, y_max))

        if self.transform:
            cropped_image = self.transform(cropped_image)

        label = torch.tensor(label, dtype=torch.long)

        return cropped_image, label


## Création du dataset

In [8]:
import os
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from statistics import mean
from PIL import Image

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms, models

In [9]:
img_dir = IMAGES_FOLDER
csv_file = "/content/drive/MyDrive/projet_collembolles/info_data.csv"  # chemin complet vers le fichier CSV
label_csv = "/content/drive/MyDrive/projet_collembolles/final_predict_logit-3.csv"  # chemin complet vers le fichier CSV

image_size = 224
BATCH_SIZE = 8
LR = 1e-4
num_classes = 9
EPOCHS = 20

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize((image_size, image_size), antialias=True),
    transforms.ToTensor(),
    # Optionally, add normalization
])


train_dataset_label = Image_label_Dataset(images_dir=img_dir, csv_file=csv_file, fallback_csv = label_csv, transform=transform, split='train')
test_dataset_label = Image_label_Dataset(images_dir=img_dir, csv_file=csv_file, fallback_csv = label_csv, transform=transform, split='test')

train_background = BackgroundDataset(img_dir, csv_file, transform=transform, num_samples=int(mean(train_dataset_label.class_counts.values())))
test_background = BackgroundDataset(img_dir, csv_file, transform=transform, num_samples=int(mean(test_dataset_label.class_counts.values())))

train_data_background = torch.utils.data.ConcatDataset([train_dataset_label, train_background])
test_data_background = torch.utils.data.ConcatDataset([test_dataset_label, test_background])

full_data_background = torch.utils.data.ConcatDataset([train_data_background, test_data_background])
full_dataset = torch.utils.data.ConcatDataset([train_dataset_label, test_dataset_label])

In [10]:
print("Taille de train_dataset_label :", len(train_dataset_label))
print("Taille de test_dataset_label :", len(test_dataset_label))
print("Taille de train_background :", len(train_background))
print("Taille de test_background :", len(test_background))
print("Taille de train_data_background :", len(train_data_background))
print("Taille de test_data_background :", len(test_data_background))
print("Taille de full_data_background :", len(full_data_background))
print("Taille de full_dataset :", len(full_dataset))

Taille de train_dataset_label : 837
Taille de test_dataset_label : 559
Taille de train_background : 104
Taille de test_background : 69
Taille de train_data_background : 941
Taille de test_data_background : 628
Taille de full_data_background : 1569
Taille de full_dataset : 1396


In [11]:
train_loader = DataLoader(train_data_background, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data_background, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 3. Chargement et adaptation du modèle ViT pré-entraîné


In [12]:
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn.metrics import f1_score
from torch.utils.tensorboard import SummaryWriter

In [13]:
model = models.vit_b_16(pretrained=True)
# Remplacer la dernière couche (head) afin d'avoir num_classes sorties
model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)

# Utilisation du GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.5, patience=3, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [15]:
from sklearn.metrics import f1_score, classification_report

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    all_train_labels = []
    all_train_preds = []

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_train_labels.extend(labels.cpu().numpy())
        all_train_preds.extend(predicted.cpu().numpy())

    train_loss /= total
    train_accuracy = correct / total
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro')

    # Évaluation
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    all_test_labels = []
    all_test_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

            all_test_labels.extend(labels.cpu().numpy())
            all_test_preds.extend(predicted.cpu().numpy())

    test_loss /= total_test
    test_accuracy = correct_test / total_test
    test_f1 = f1_score(all_test_labels, all_test_preds, average='macro')

    scheduler.step(test_loss)

    print(f"\nEpoch [{epoch+1}/{EPOCHS}]")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Train F1: {train_f1:.4f}")
    print("Train classification report :")
    print(classification_report(all_train_labels, all_train_preds, digits=4))

    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}, Test F1: {test_f1:.4f}")
    print("Test classification report :")
    print(classification_report(all_test_labels, all_test_preds, digits=4))


Epoch [1/20]
Train Loss: 1.5313, Train Acc: 0.4516, Train F1: 0.3228
Train classification report :
              precision    recall  f1-score   support

           0     0.4286    0.7213    0.5377       287
           1     0.4737    0.5870    0.5243       138
           2     0.3077    0.0533    0.0909        75
           3     0.4815    0.4062    0.4407        64
           4     0.1613    0.1000    0.1235        50
           5     0.3125    0.1724    0.2222        87
           6     0.3810    0.1231    0.1860        65
           7     0.3750    0.0423    0.0759        71
           8     0.6786    0.7308    0.7037       104

    accuracy                         0.4516       941
   macro avg     0.4000    0.3263    0.3228       941
weighted avg     0.4245    0.4516    0.4015       941

Test Loss: 1.5771, Test Acc: 0.4283, Test F1: 0.3491
Test classification report :
              precision    recall  f1-score   support

           0     0.3526    0.6387    0.4544       191
    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch [2/20]
Train Loss: 1.0043, Train Acc: 0.6206, Train F1: 0.5657
Train classification report :
              precision    recall  f1-score   support

           0     0.5333    0.7247    0.6145       287
           1     0.7482    0.7536    0.7509       138
           2     0.4182    0.3067    0.3538        75
           3     0.7627    0.7031    0.7317        64
           4     0.3235    0.2200    0.2619        50
           5     0.5733    0.4943    0.5309        87
           6     0.6667    0.5538    0.6050        65
           7     0.5172    0.2113    0.3000        71
           8     0.9340    0.9519    0.9429       104

    accuracy                         0.6206       941
   macro avg     0.6086    0.5466    0.5657       941
weighted avg     0.6161    0.6206    0.6071       941

Test Loss: 1.0778, Test Acc: 0.6051, Test F1: 0.5422
Test classification report :
              precision    recall  f1-score   support

           0     0.6051    0.4974    0.5460       191
    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch [3/20]
Train Loss: 0.8685, Train Acc: 0.6674, Train F1: 0.6132
Train classification report :
              precision    recall  f1-score   support

           0     0.6140    0.7317    0.6677       287
           1     0.7763    0.8551    0.8138       138
           2     0.5000    0.3600    0.4186        75
           3     0.8361    0.7969    0.8160        64
           4     0.2581    0.1600    0.1975        50
           5     0.6395    0.6322    0.6358        87
           6     0.5915    0.6462    0.6176        65
           7     0.6279    0.3803    0.4737        71
           8     0.8911    0.8654    0.8780       104

    accuracy                         0.6674       941
   macro avg     0.6372    0.6031    0.6132       941
weighted avg     0.6574    0.6674    0.6566       941

Test Loss: 0.8149, Test Acc: 0.6943, Test F1: 0.6285
Test classification report :
              precision    recall  f1-score   support

           0     0.5846    0.7958    0.6741       191
    

# TRAIN FULL

In [15]:
EPOCHS = 50

In [16]:
full_loader = DataLoader(full_data_background, batch_size=BATCH_SIZE, shuffle=True)

In [17]:
model = None
model = models.vit_b_16(pretrained=True)

model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)

# Utilisation du GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',
                                                       factor=0.5, patience=3, verbose=True)

In [19]:
def full_train(model_vit, full_loader, criterion, optimizer, EPOCHS=50):
    best_f1 = 0.0  # Meilleur F1 score
    best_model_wts = None

    for epoch in range(EPOCHS):
        model_vit.train()  # Passer en mode entraînement
        running_loss = 0.0
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []

        epoch_iterator = tqdm(full_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch")

        for inputs, labels in epoch_iterator:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model_vit(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        # Calcul des métriques
        epoch_loss = running_loss / len(full_loader)
        epoch_acc = 100 * correct / total
        epoch_f1 = f1_score(all_labels, all_predictions, average="macro")

        # Affichage des métriques
        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f} | F1 Score: {epoch_f1:.4f}")

        # Si le F1 score est le meilleur, sauvegarder le modèle
        if epoch_f1 > best_f1:
            best_f1 = epoch_f1
            best_model_wts = model_vit.state_dict()

        # Mettre à jour le scheduler
        scheduler.step(epoch_f1)

    # Sauvegarde du meilleur modèle basé sur F1 score
    model_vit.load_state_dict(best_model_wts)
    run_name = f"VITAXEL_NET_MIN3_NEWLOSS_{BATCH_SIZE}BATCH_{LR}LR_{EPOCHS}EPOCH_{image_size}SIZE"
    torch.save(model_vit.state_dict(), f"{run_name}.pth")
    print(f"Training complete. Best F1 Score: {best_f1:.4f}")

full_train(model, full_loader, criterion, optimizer, EPOCHS)

Epoch 1/50: 100%|██████████| 197/197 [07:51<00:00,  2.39s/batch]


Epoch 1/50 - Loss: 1.4206 - Acc: 50.2231 | F1 Score: 0.3940


Epoch 2/50: 100%|██████████| 197/197 [01:22<00:00,  2.40batch/s]


Epoch 2/50 - Loss: 0.9807 - Acc: 63.7986 | F1 Score: 0.5898


Epoch 3/50: 100%|██████████| 197/197 [01:23<00:00,  2.35batch/s]


...

Epoch 48/50 - Loss: 0.0006 - Acc: 100.0000 | F1 Score: 1.0000


Epoch 49/50: 100%|██████████| 197/197 [01:22<00:00,  2.39batch/s]


Epoch 49/50 - Loss: 0.0010 - Acc: 100.0000 | F1 Score: 1.0000


Epoch 50/50: 100%|██████████| 197/197 [01:22<00:00,  2.38batch/s]


Epoch 50/50 - Loss: 0.0008 - Acc: 100.0000 | F1 Score: 1.0000
Training complete. Best F1 Score: 1.0000


In [20]:
run_name = f"VIT_MIN4_NEWLABEL_CNNVIT_{BATCH_SIZE}BATCH_{LR}LR_{EPOCHS}EPOCH_{image_size}SIZE"
torch.save(model.state_dict(), f"{run_name}.pth")

# Inference

In [21]:
model.eval()

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [22]:
model = None
model = models.vit_b_16(pretrained=True)

model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)
# à changer
model.to(device)
run_name = "/content/drive/MyDrive/projet_collembolles/VIT_MIN4_NEWLABEL_CNNVIT_8BATCH_0.0001LR_50EPOCH_224SIZE.pth"
model.load_state_dict(torch.load(f'{run_name}'))
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [23]:
class CustomTestDataset(Dataset):
    def __init__(self, images_dir, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.images_dir, image_name)
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        image_name = os.path.splitext(image_name)[0]

        return image, image_name

In [24]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size), antialias=True),
    transforms.ToTensor(),
    ])

In [25]:
test_img_dir = '/content/drive/MyDrive/projet_collembolles/datatest'

# Créer un Dataset pour le test
test_dataset = CustomTestDataset(images_dir=test_img_dir, transform=transform)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"Nombre d'images dans le DataLoader de test: {len(test_loader.dataset)}")

Nombre d'images dans le DataLoader de test: 1344


In [27]:
import csv
results = []

# Assurer que le modèle est en mode évaluation
model.eval()

# Désactiver la mise à jour des gradients
with torch.no_grad():
    for idx, (inputs, image_names) in enumerate(tqdm(test_loader, desc="Testing")):
        inputs = inputs.to(device)

        # Prédictions du modèle
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Ajouter au résultat (nom de l'image sans extension et la classe prédite)
        for i in range(len(inputs)):
            # On s'assure que le nom de l'image est correctement formaté sans tuple
            image_name = image_names[i]  # Récupérer le nom du fichier de l'image

            # Ajouter les résultats
            predicted_class = predicted[i].item()
            results.append([image_name, predicted_class])

# Sauvegarder les résultats dans un fichier CSV
csv_output_path = '/content/drive/MyDrive/projet_collembolles/VIT_MIN4_NEWLABEL_CNNVIT.csv'
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['idx', 'gt'])  # Écrire l'en-tête
    writer.writerows(results)  # Écrire les lignes des résultats

print(f"Fichier CSV de soumission sauvegardé sous {csv_output_path}")


Testing: 100%|██████████| 168/168 [00:41<00:00,  4.02it/s]

Fichier CSV de soumission sauvegardé sous /content/drive/MyDrive/projet_collembolles/VIT_MIN4_NEWLABEL_CNNVIT.csv


In [28]:
import csv
results = []
import torch.nn.functional as F

# Assurer que le modèle est en mode évaluation
model.eval()

# Désactiver la mise à jour des gradients
with torch.no_grad():
    for idx, (inputs, image_names) in enumerate(tqdm(test_loader, desc="Testing")):
        inputs = inputs.to(device)

        # Prédictions du modèle
        outputs = model(inputs)
        probas_axel = F.softmax(outputs, dim=1)

        # Ajouter au résultat (nom de l'image sans extension et la classe prédite)
        for i in range(len(inputs)):
            # On s'assure que le nom de l'image est correctement formaté sans tuple
            image_name = image_names[i]  # Récupérer le nom du fichier de l'image

            # Ajouter les résultats
            predicted = probas_axel[i].tolist()
            results.append([image_name, predicted])

# Sauvegarder les résultats dans un fichier CSV
csv_output_path = '/content/drive/MyDrive/projet_collembolles/VIT_MIN4_NEWLABEL_CNNVIT_softmax.csv'
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['idx', 'gt'])  # Écrire l'en-tête
    writer.writerows(results)  # Écrire les lignes des résultats

print(f"Fichier CSV de soumission sauvegardé sous {csv_output_path}")

Testing: 100%|██████████| 168/168 [00:17<00:00,  9.58it/s]

Fichier CSV de soumission sauvegardé sous /content/drive/MyDrive/projet_collembolles/VIT_MIN4_NEWLABEL_CNNVIT_softmax.csv
